In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
# import pysqlite3
# import sys
# sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
from langchain_chroma import Chroma

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import pymupdf

In [3]:
loader = PyMuPDFLoader("book/nbc2020_p1.pdf",extract_images=True)
pages = loader.load_and_split()

In [15]:
pages[0]

Document(page_content='BUILDING\n  National Building Code of Canada 2020 \t\nVolume 1\nCANADIAN COMMISSION ON \nBUILDING AND FIRE CODES', metadata={'source': 'book/nbc2020_p1.pdf', 'file_path': 'book/nbc2020_p1.pdf', 'page': 0, 'total_pages': 1530, 'format': 'PDF 1.7', 'title': 'National Building Code of Canada 2020, 1st printing', 'author': 'NRC-CNRC', 'subject': 'National Building Code of Canada 2020', 'keywords': '', 'creator': 'PScript5.dll Version 5.2.2', 'producer': 'Acrobat Distiller 15.0 (Windows)', 'creationDate': "D:20220323135647-04'00'", 'modDate': "D:20220923095111-04'00'", 'trapped': ''})

In [ ]:
[doc.page_content.replace('\n', ' ')  for doc in pages]

In [5]:
doc = pymupdf.Document("book/nbc2020_p1.pdf")

In [55]:
doc.load_page(7).get_text()

"Preface\nMore information on the Codes development process is available on the NRC's website.\nNational Building Code of Canada 2020\nThe NBC sets out technical provisions for the design and construction of new buildings. It\nalso applies to the alteration, change of use and demolition of existing buildings.\nThe NBC establishes requirements to address the following five objectives:\n• safety\n• health\n• accessibility\n• fire and structural protection of buildings\n• environment\nCode provisions do not necessarily address all the characteristics of buildings that might be\nconsidered to have a bearing on the Code's objectives. Through the extensive consensus\nprocess used to develop and maintain the National Model Codes (see the section entitled\nDevelopment of the National Model Codes), the Codes-user community has decided which\ncharacteristics should be regulated through the NBC.\nThe provisions of the NBC can be considered as the minimum acceptable measures required\nto adequatel

In [46]:
doc.metadata

{'format': 'PDF 1.7',
 'title': 'National Building Code of Canada 2020, 1st printing',
 'author': 'NRC-CNRC',
 'subject': 'National Building Code of Canada 2020',
 'keywords': '',
 'creator': 'PScript5.dll Version 5.2.2',
 'producer': 'Acrobat Distiller 15.0 (Windows)',
 'creationDate': "D:20220323135647-04'00'",
 'modDate': "D:20220923095111-04'00'",
 'trapped': '',
 'encryption': None}

In [6]:
doc_toc = doc.get_toc()

In [7]:
doc_toc

[[1, 'National Building Code of Canada 2020', 2],
 [2, 'Volume I', 6],
 [3, 'Preface', 6],
 [4, 'National Building Code of Canada 2020', 7],
 [4, 'Relationship between the NBC and the NFC', 7],
 [4, 'Relationship between the NBC and the NECB', 8],
 [4, 'Code Requirements', 8],
 [4, 'Objective-Based Code Format', 8],
 [4, 'Objectives', 9],
 [4, 'Functional Statements', 9],
 [4, 'Intent Statements', 9],
 [4, 'Structure of the NBC', 10],
 [4, 'Division A: Compliance, Objectives and Functional Statements', 10],
 [4, 'Division B: Acceptable Solutions', 10],
 [4, 'Division C: Administrative Provisions', 10],
 [4, 'Relationship between Division A and Division B', 10],
 [4, 'Parts in Division B and Professional Disciplines', 11],
 [4, "What's New in the 2020 Edition", 11],
 [4, 'New Part 2', 11],
 [4, 'Additional Information', 11],
 [4, 'Numbering System', 11],
 [4,
  'Meaning of the Words “And” and “Or” between the Clauses and Subclauses of a Sentence',
  12],
 [4, 'Change Indication', 12],
 

In [8]:
class Document:
    def __init__(self, content, metadata):
        self.content = content
        self.metadata = metadata

In [59]:
import re

def extract_text(page_number, stop_title=None):
    # Find the document page that matches the given page_number in the metadata
    for page in pages:
        if page.metadata['page'] == page_number:
            content = page.page_content.replace('\n', ' ')
            # If a stop_title is provided, search for it in the content and slice up to that point
            if stop_title:
                # Create a pattern to locate the title, escaping special characters and considering case insensitivity
                pattern = re.escape(stop_title)
                match = re.search(pattern, content, re.IGNORECASE)
                if match:
                    return content[:match.start()]  # Return content up to the start of the matched title
            return content
    return ""

section_contents = []

for i in range(len(doc_toc)):
    current_page_number = doc_toc[i][2]
    current_title = doc_toc[i][1]
    if i+1 < len(doc_toc):
        next_page_number = doc_toc[i+1][2]
        next_title = doc_toc[i+1][1]
    else:
        next_page_number = current_page_number + 1  # Assuming end of document if this is the last entry
        next_title = None

    section_content = ""
    # Extract content from the current page number up to the page before the next TOC entry
    if next_page_number > current_page_number:
        for page_num in range(current_page_number, next_page_number):
            section_content += extract_text(page_num, next_title if page_num == next_page_number - 1 else None)
    else:
        # If the next entry is on the same page number, just extract the current page's content up to the title
        section_content = extract_text(current_page_number-1,next_title)

    section_contents.append({
        "title": current_title,
        "content": section_content
    })
    # if i >2:
    #     break

In [29]:
for page in pages:
    if page.metadata['page'] == current_page_number:
        content = page.page_content.replace('\n', ' ')
        print(content)

Preface • limitations on hazardous contents in and around buildings • the establishment of fire safety plans • fire safety at construction and demolition sites In addition, the NFC contains provisions regarding fire safety and fire protection features that must be added to existing buildings when certain hazardous activities or processes are introduced into these buildings. Some of the NFC's provisions are incorporated by reference in the NBC and, thus, may apply to original construction, alterations, or changes in use. Relationship between the NBC and the NECB The provisions in Section 9.36. of Division B of the NBC are tied to the environment objective. These provisions, which apply to housing and small buildings, have a similar scope to that of the NECB, except that they do not address lighting and electrical power systems. The NECB is referenced in NBC Section 9.36. as an acceptable solution. Code Requirements Most NBC requirements address at least one of the Code's five stated obj

In [58]:
section_contents

[{'title': 'National Building Code of Canada 2020',
  'content': "First Edition 1941 Second Edition 1953 Third Edition 1960 Fourth Edition 1965 Fifth Edition 1970 Sixth Edition 1975 Seventh Edition 1977 Eighth Edition 1980 Ninth Edition 1985 Tenth Edition 1990 Eleventh Edition 1995 Twelfth Edition 2005 Thirteenth Edition 2010 Fourteenth Edition 2015 Fifteenth Edition 2020 Paper: ISBN 978-0-660-37913-5 NR24-28/2020E PDF: ISBN 978-0-660-37912-8 NR24-28/2020E-PDF NRCC-CONST-56435E © His Majesty the King in Right of Canada,  as represented by the National Research  Council of Canada, 2022 Printed in Canada First Printing Aussi disponible en français : Code national du bâtiment – Canada 2020 NRCC-CONST-56435F Papier : ISBN 978-0-660-37915-9 PDF : ISBN 978-0-660-37914-2 © His Majesty the King in Right of Canada, as represented by the National Research Council of Canada, 2022 © Sa Majesté le Roi du chef du Canada, représenté par le Conseil national de recherches du Canada, 2022Table of Conten

In [90]:
docs = [entry['content'].replace('\n', ' ') for entry in section_contents]
titles = [entry['title'] for entry in section_contents]
# If you need to remove other non-readable characters or multiple types of whitespace, you could use re.sub()
# For example, to also remove tabs or multiple spaces, you might do:
# docs = [re.sub(r'\s+', ' ', entry['content']) for entry in section_contents]

In [86]:
__import__('pysqlite3')
import pysqlite3
sys.modules['sqlite3'] = sys.modules["pysqlite3"]
import chromadb
persistent_client = chromadb.PersistentClient(path="NBC")
collection = persistent_client.get_or_create_collection("nbc")

In [88]:
titles

['National Building Code of Canada 2020',
 'Volume I',
 'Preface',
 'National Building Code of Canada 2020',
 'Relationship between the NBC and the NFC',
 'Relationship between the NBC and the NECB',
 'Code Requirements',
 'Objective-Based Code Format',
 'Objectives',
 'Functional Statements',
 'Intent Statements',
 'Structure of the NBC',
 'Division A: Compliance, Objectives and Functional Statements',
 'Division B: Acceptable Solutions',
 'Division C: Administrative Provisions',
 'Relationship between Division A and Division B',
 'Parts in Division B and Professional Disciplines',
 "What's New in the 2020 Edition",
 'New Part 2',
 'Additional Information',
 'Numbering System',
 'Meaning of the Words “And” and “Or” between the Clauses and Subclauses of a Sentence',
 'Change Indication',
 'Units',
 'Complementary Publications',
 'Commercial Reproduction',
 'Contact Information',
 "The development of many provisions in the NBC and the assessment of conformity to those provisions are sup

In [92]:
collection.add(
    ids=[str(i) for i in range(len(docs))],
    documents=[doc for doc in docs],
    metadatas=[{"tilte": title} for title in titles]
)

In [93]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
# db = Chroma.from_documents(pages, embedding_function,persist_directory="./chroma_db")

/mnt/sda1/projects/Nam_exp/RAG_for_book/newlife/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [70]:
langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="nbc",
    embedding_function=embedding_function,
)

In [71]:
print("There are", langchain_chroma._collection.count(), "in the collection")

There are 2880 in the collection


In [82]:
# db = Chroma(persist_directory= "./chroma_db", embedding_function=embedding_function)
type(langchain_chroma)

langchain_chroma.vectorstores.Chroma

In [80]:
# db4 = Chroma(
#     client=persistent_client,
#     collection_name="nbc",
#     embedding_function=embedding_function,
# )
query = "Roofs with Solar Panels"
docs = langchain_chroma.similarity_search(query)
print(docs[0].page_content)

Division B 4.1.7.1. ii) S shall be taken as 0.0 on the panels, and iii) S for all roof areas shall be taken as the sum of S on the panels, as derived from Subclause (a)(i) and shifted by a distance of wp downslope onto the roof, where wp is the panel width along the roof slope, and S on the roof areas, as derived from Subclauses (a)(ii) to (a)(iv) (see Note A-4.1.6.16.(5)(b)). 6) For flat roofs with Tilted solar panels, the snow loads, S, shall be determined in accordance with the requirements for roofs without solar panels, except that a) Ca shall be taken as 0.0 for the panels, b) Ca shall be taken as 1.0 for roof areas beyond a distance of 5(h – CbCwSs/γ) from the lowest edge of the panels, where h is the height of the highest edge of the panels above the roof surface, c) except as provided in Clauses (d) and (e), for roof areas within a distance of 5(h – CbCwSs/γ) from the lowest edge of the panels, Ca shall be taken as i) 1.25 for (hg – CbCwSs/γ) ≤0.3 m, where hg is the gap height

In [10]:
from langchain.vectorstores import Chroma

In [6]:
# query it
retriever = db.as_retriever(search_kwargs={"k": 2})

In [5]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
import os
import textwrap
from IPython.display import Markdown
load_dotenv()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

NameError: name 'load_dotenv' is not defined

In [4]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [11]:
import streamlit as st
import os
from dotenv import load_dotenv

import google.generativeai as genai
from util import make_prompt
from database import DocumentDatabase
# import pysqlite3
# import sys
# sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
from langchain_chroma import Chroma


# Load environment variables
load_dotenv()

# Configure Google Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Initialize the database
db = DocumentDatabase(persist_directory="./chroma_db")
db.load_database()

d:\Project\RAG_for_book\rag_evn\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
query="what is database"
db.similarity_search(query=query,k=10)

[Document(page_content='Chapter 1\nThe Worlds of Database  \nSystems\nDatabases today are essential to every business. Whenever you visit a major \nWeb site — Google, Yahoo!, Amazon.com, or thousands of smaller sites that \nprovide information — there is a database behind the scenes serving up the \ninformation you request. Corporations maintain all their important records in \ndatabases. Databases are likewise found at the core of many scientific investi\xad\ngations. They represent the data gathered by astronomers, by investigators of \nthe human genome, and by biochemists exploring properties of proteins, among \nmany other scientific activities.\nThe power of databases comes from a body of knowledge and technology \nthat has developed over several decades and is embodied in specialized soft\xad\nware called a database management system,  or DBMS , or more colloquially a \n“database system.” A DBMS is a powerful tool for creating and managing large \namounts of data efficiently and 

In [14]:
db.db.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021C87EC5A90>)

In [8]:
page

In [ ]:
vectorstore.as_retriever()

In [4]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_documents(pages)
bm25_retriever.k =  10  # Retrieve top 2 results
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.1, 0.8]
)

In [28]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [29]:
context = docs[0].page_content

prompt = make_prompt(query, context)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'what is Modifying Relation Schemas'
  PASSAGE: '2.3. DEFINING  A RELATION SCHEMA IN  SQL 33 The gender attribute has values that are a single letter, M  or F. Thus, we  can safely use a single character as the type of this attribute. Finally, the  b irth d a te  attribute naturally deserves the data type D A TE. □ 2.3.4 Modifying Relation Schemas We now know how to declare a table. But what if we need to change the schema  of the table after it has been in use for a long time and has many tuples in its  current instance? We can remove the

In [5]:
query = "what is Modifying Relation Schemas"
docs = db.similarity_search(query)
docs

NameError: name 'db' is not defined

In [39]:
docs[0].metadata

{'page': 69, 'source': 'book/ullman_the_complete_book.pdf'}

In [31]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')
answer = model.generate_content(prompt)
answer

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Modifying a relation schema, or table definition, in SQL involves making changes to the existing schema, such as adding or deleting attributes.   To add an attribute, the ALTER TABLE statement can be used with the ADD keyword, followed by the attribute name and data type.   For example, to add a phone attribute to the MovieStar table as a fixed-length string of 16 bytes, the following statement would be used: ALTER TABLE MovieStar ADD phone CHAR(16).   Similarly, to delete an attribute, the ALTER TABLE statement can be used with the DROP keyword, followed by the attribute name.   For example, to delete the birthdate attribute from the MovieStar table, the following statement would be used: ALTER TABLE MovieStar DROP birthdate.   These modifications allow you to

In [34]:
Markdown(answer.text)

Modifying a relation schema, or table definition, in SQL involves making changes to the existing schema, such as adding or deleting attributes.   To add an attribute, the ALTER TABLE statement can be used with the ADD keyword, followed by the attribute name and data type.   For example, to add a phone attribute to the MovieStar table as a fixed-length string of 16 bytes, the following statement would be used: ALTER TABLE MovieStar ADD phone CHAR(16).   Similarly, to delete an attribute, the ALTER TABLE statement can be used with the DROP keyword, followed by the attribute name.   For example, to delete the birthdate attribute from the MovieStar table, the following statement would be used: ALTER TABLE MovieStar DROP birthdate.   These modifications allow you to change the schema of a table without having to recreate it from scratch.

In [30]:

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [1]:
pip install -U sentence-transformers

  Using cached sentence_transformers-3.0.0-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.0.0-py3-none-any.whl (224 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.7.0
    Uninstalling sentence-transformers-2.7.0:
      Successfully uninstalled sentence-transformers-2.7.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sentence_transformers import CrossEncoder

# Load the model, here we use our base sized model
model = CrossEncoder("mixedbread-ai/mxbai-rerank-base-v1")


# Example query and documents
query = "Who wrote 'To Kill a Mockingbird'?"
documents = [
    "'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature.",
    "The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil.",
    "Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961.",
    "Jane Austen was an English novelist known primarily for her six major novels, which interpret, critique and comment upon the British landed gentry at the end of the 18th century.",
    "The 'Harry Potter' series, which consists of seven fantasy novels written by British author J.K. Rowling, is among the most popular and critically acclaimed books of the modern era.",
    "'The Great Gatsby', a novel written by American author F. Scott Fitzgerald, was published in 1925. The story is set in the Jazz Age and follows the life of millionaire Jay Gatsby and his pursuit of Daisy Buchanan."
]

# Lets get the scores
results = model.rank(query, documents, return_documents=True, top_k=3)


d:\Project\RAG_for_book\rag_evn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Project\RAG_for_book\rag_evn\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Project\RAG_for_book\rag_evn\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kinhd\.cache\huggingface\hub\models--mixedbread-ai--mxbai-rerank-base-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by

In [3]:
results

[{'corpus_id': 0,
  'score': 0.9968497,
  'text': "'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature."},
 {'corpus_id': 2,
  'score': 0.99251467,
  'text': "Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961."},
 {'corpus_id': 1,
  'score': 0.2528575,
  'text': "The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil."}]

In [15]:
query = "What is Data Redundancy?"


# Retrieve relevant documents/products
docs = ensemble_retriever.get_relevant_documents(query)

# Extract and print only the page content from each document
# for doc in docs:
#     print(doc.page_content)

docs

[Document(page_content='590 CHAPTER 13. SECONDARY STORAGE M ANAGEM ENT\na) What are the corresponding blocks of the redundant disks?\nb) If the third disk’s block is rewritten to be 10000000, what steps must be \ntaken to change other disks?\nE xercise 13.4.10: Describe the steps taken to recover from the following fail\xad\nures using the RAID level 6 scheme with seven disks: (a) disks 1 and 7, (b) disks \n1 and 4, (c) disks 3 and 6.\n13.5 Arranging D ata on Disk\nWe now turn to the m atter of how disks are used store databases. A data \nelement such as a tuple or object is represented by a record,  which consists of \nconsecutive bytes in some disk block. Collections such as relations are usually \nrepresented by placing the records that represent their data elements in one or \nmore blocks. It is normal for a disk block to hold only elements of one relation, \nalthough there are organizations where blocks hold tuples of several relations. \nIn this section, we shall cover the basic 

In [1]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("Cohere/rerank-multilingual-v3.0")
text = "Hello World, this is my input string!"
enc = tokenizer.encode(text)

/mnt/sda1/projects/Nam_exp/RAG_for_book/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print("Encoded input:")
print(enc.ids)

print("Tokens:")
print(enc.tokens)

number_of_tokens = len(enc.ids)
print("Number of tokens:", number_of_tokens)

Encoded input:
[1, 30274, 4026, 262, 715, 340, 1038, 13038, 11307, 310, 2]
Tokens:
['[CLS]', '▁Hello', '▁World', ',', '▁this', '▁is', '▁my', '▁input', '▁string', '!', '[SEP]']
Number of tokens: 11


In [1]:
!pip install -U FlagEmbedding

     |████████████████████████████████| 141 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 779.1 MB 45 kB/s  eta 0:00:012    |█████▋                          | 136.9 MB 1.8 MB/s eta 0:06:04     |██████████████████▊             | 454.7 MB 3.7 MB/s eta 0:01:28     |█████████████████████████▉      | 629.6 MB 4.3 MB/s eta 0:00:35     |██████████████████████████▎     | 640.3 MB 3.6 MB/s eta 0:00:40
     |████████████████████████████████| 9.1 MB 507 kB/s eta 0:00:01
     |████████████████████████████████| 542 kB 780 kB/s eta 0:00:01
     |████████████████████████████████| 302 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 1.3 MB/s eta 0:00:01
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64

In [2]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score)

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores)

/mnt/sda1/projects/Nam_exp/RAG_for_book/rag1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named '_bz2'